In [71]:
#Question 2.2 d)
from automata.fa.nfa import NFA

#first create a map of all input set of 3 digits
#this allows us to use letters such as a,b,c... instead of number strings, which nfa can only read
#individually
map_AddBin3 = {'000':'a', '100':'b', '010':'c', '001':'d', '110':'e', '011':'f', '101':'g', '111':'h'}
nfa_AddBin3 = NFA(
    states={'q0', 'q1', 'q2'},
    input_symbols={'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', },
    transitions={
        'q0': {'a': {'q0'}, 'g': {'q0'}, 'f': {'q0'}, 'd': {'q1'}},
        'q1': {'b': {'q1'}, 'c': {'q1'}, 'h': {'q1'}, 'e': {'q0', 'q2'}},
        'q2': {'a': {'q0', 'q1'}, 'g': {'q0', 'q1'}, 'f': {'q0', 'q1'}, 'd': {'q1'}}
    },
    initial_state='q2',
    final_states={'q0'}
)

#convert list of strings into characters to be validated by nfa_AddBin3
input_string_converted = ''

#multiple test cases
input_string = ['001', '100', '110']
#input_string = ['101', '011']
#input_string = ['111', '101', '110']
for i in input_string:
    input_string_converted += map_AddBin3[i]

print(input_string_converted)
print(nfa_AddBin3.read_input(input_string_converted))
print(nfa_AddBin3.accepts_input(input_string_converted))

dbe
{'q2', 'q0'}
True


In [72]:
#Question 3.3. Refer to assignment pdf for finite state diagram
#In order to avoid a bunch of states in one NFA, I instead left them all as individuals
#The first nfa, nfa_path, will read the first three characters of a given input string
#This will help find the next operator to use, if so available
#If available, a final function then directs remaining string into appropriate nfa for that operator

#Codes for each operator is as follows: 
#Addbin3 (011), OR (100), X-OR(101), AND (110), and COMPLEMENT (111),
nfa_path = NFA(
    states={'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'COMP', 'AND', 'OR', 'XOR', 'AB3'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'1': {'q1'}, '0': {'q4'}},
        'q1': {'1': {'q2'}, '0': {'q3'}},
        'q2': {'1': {'COMP'}, '0': {'AND'}},
        'q3': {'1': {'XOR'}, '0': {'OR'}},
        'q4': {'1': {'q5'}},
        'q5': {'1': {'AB3'}},
        'COMP': {},
        'AND': {},
        'OR': {},
        'XOR': {},
        'AB3': {}
    },
    initial_state='q0',
    final_states={'COMP', 'AND', 'OR', 'XOR', 'AB3'}
)

In [73]:
nfa_AND = NFA(
    states={'q0', 'q1', 'q2', 'q3', 'q4', 'q5'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'1': {'q2'}, '0': {'q1'}},
        'q1': {'1': {'q3'}, '0': {'q3'}},
        'q2': {'1': {'q4'}, '0': {'q3'}},
        'q3': {'0': {'q5'}},
        'q4': {'1': {'q5'}},
        'q5': {}
    },
    initial_state='q0',
    final_states={'q5'}
)

In [74]:
nfa_OR = NFA(
    states={'q0', 'q1', 'q2', 'q3', 'q4', 'q5'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'1': {'q2'}, '0': {'q1'}},
        'q1': {'1': {'q4'}, '0': {'q3'}},
        'q2': {'1': {'q4'}, '0': {'q4'}},
        'q3': {'0': {'q5'}},
        'q4': {'1': {'q5'}},
        'q5': {}
    },
    initial_state='q0',
    final_states={'q5'}
)

In [75]:
nfa_XOR = NFA(
    states={'q0', 'q1', 'q2', 'q3', 'q4', 'q5'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'1': {'q2'}, '0': {'q1'}},
        'q1': {'1': {'q4'}, '0': {'q3'}},
        'q2': {'1': {'q3'}, '0': {'q4'}},
        'q3': {'0': {'q5'}},
        'q4': {'1': {'q5'}},
        'q5': {}
    },
    initial_state='q0',
    final_states={'q5'}
)

In [76]:
nfa_COMP = NFA(
    states={'q0', 'q1', 'q2', 'q3'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'1': {'q2'}, '0': {'q1'}},
        'q1': {'1': {'q3'}},
        'q2': {'0': {'q3'}},
        'q3': {}
    },
    initial_state='q0',
    final_states={'q3'}
)

In [77]:
#This is the big final function I was referring to. Does all the work to ensure everything runs
def read_string(input_string):
    
    #split input string into first 3 characters, then the rest
    path_string = input_string[:3]
    operator_string = input_string[3:]
    #print(operator_string)
    accept = None
    if nfa_path.accepts_input(path_string):
        operator = list(nfa_path.read_input(path_string))[0]
    if operator == "COMP":
        accept = nfa_COMP.accepts_input(operator_string)
    if operator == "AND":
        accept = nfa_AND.accepts_input(operator_string)
    if operator == "OR":
        accept = nfa_OR.accepts_input(operator_string)
    if operator == "XOR":
        accept = nfa_XOR.accepts_input(operator_string)
        
    #some more heavy lifting needs to be done for AB3 since we defined it earlier as a list of strings
    if operator == "AB3" and (len(operator_string)%3 == 0):
        temp_list = []
        map_AddBin3 = {'000':'a', '100':'b', '010':'c', '001':'d', '110':'e', '011':'f', '101':'g', '111':'h'}
        for i in range(len(operator_string)//3):
            temp_str = ''
            for j in range(3):
                digit = (i*3)+j
                temp_str += (operator_string[digit])
            temp_list.append(temp_str)
        input_string_converted = ''
        for k in temp_list:
            input_string_converted += map_AddBin3[k]
        accept = nfa_AddBin3.accepts_input(input_string_converted)
    return accept

In [78]:
#tests each string to make sure everything works, as you can see all tests pass

assert(read_string("11101"))
assert(read_string("100101"))
assert(read_string("110111"))
assert(read_string("011001100110"))